In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import math
plt.style.use('fivethirtyeight')

In [ ]:
df = pd.read_csv('/kaggle/input/tesla-stock-price/Tesla.csv - Tesla.csv.csv')
df


In [ ]:
df.info()

**Visualizing the Data**

In [ ]:
plt.figure(figsize=(13,5))
plt.plot(df['Adj Close'], label = 'TSLA')
plt.title('Tesla Adjusted Close Price History')
plt.xlabel('Year')
plt.ylabel('Adj. Close Price in USD')
plt.legend(loc='upper left')
plt.show()



# **Simple Moving Average - 30 Day Average**

In [ ]:
SMA30 = pd.DataFrame()
SMA30['Adj Close'] = df['Adj Close'].rolling(window=30).mean()
SMA30

# **Simple Moving Average - 100 Day Average**

In [ ]:
SMA100 = pd.DataFrame()
SMA100['Adj Close'] = df['Adj Close'].rolling(window=100).mean()
SMA100

**Visualzie the Data**

In [ ]:
plt.figure(figsize=(12.5,4.5))
plt.plot(df['Adj Close'], label = 'TSLA')
plt.plot(SMA30['Adj Close'], label = 'SMA30')
plt.plot(SMA100['Adj Close'], label = 'SMA100')
plt.title('Tesla Adjusted Close Price History and Simple Moving Averages')
plt.xlabel('Year')
plt.ylabel('Adj. Close Price in USD')
plt.legend(loc='upper left')
plt.show()

In [ ]:
data = pd.DataFrame()
data['TSLA'] = df['Adj Close']
data['SMA30'] = SMA30['Adj Close']
data['SMA100'] = SMA100['Adj Close']
data

# **Buy-Sell Signal**

In [ ]:
def buy_sell(data):
    sigPriceBuy = []
    sigPriceSell = []
    flag = -1
    
    for i in range(len(data)):
        if data['SMA30'][i] > data['SMA100'][i]:
            if flag != 1:
                sigPriceBuy.append(data['TSLA'][i])
                sigPriceSell.append(np.nan)
                flag = 1
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        elif data['SMA30'][i] < data['SMA100'][i]:
            if flag != 0:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(data['TSLA'][i])
                flag = 0
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        else:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
            
    return (sigPriceBuy, sigPriceSell)
            


In [ ]:
buy_sell = buy_sell(data)
data['Buy_Signal_Price'] = buy_sell[0]
data['Sell_Signal_Price'] = buy_sell[1]

In [ ]:
data

In [ ]:
plt.figure(figsize=(12.5,4.5))
plt.plot(data['TSLA'], label = 'TSLA', alpha=0.35)
plt.plot(data['SMA30'], label = 'SMA30', alpha=0.35)
plt.plot(data['SMA100'], label = 'SMA100', alpha=0.35)
plt.scatter(data.index, data['Buy_Signal_Price'], label = 'Buy', marker = '^', color = 'green')
plt.scatter(data.index, data['Sell_Signal_Price'], label = 'Sell', marker = 'v', color = 'red')
plt.title('Tesla Adjusted Close Price History with Buy/Sell Signals')
plt.xlabel('Year')
plt.ylabel('Adj. Close Price in USD')
plt.legend(loc='upper left')
plt.show()

# **Bollinger Bands Method**

In [ ]:
df1 = pd.read_csv('/kaggle/input/tesla-stock-price/Tesla.csv - Tesla.csv.csv')
df1 = df.set_index(pd.DatetimeIndex(df1['Date'].values)) #change index to date
df1

Let us calculate Simple Moving Average, Standard Deviation, Upper Band and the Lower Band

In [ ]:
period = 20
#Simple Moving Average
df1['SMA'] = df1['Close'].rolling(window=period).mean()
#Standard Deviation
df1['STD'] = df1['Close'].rolling(window=period).std()
#Upper Band
df1['Upper'] = df1['SMA'] + (df1['STD'] * 2)
#Lower Band
df1['Lower'] = df1['SMA'] - (df1['STD'] * 2)

column_list = ['Close', 'SMA', 'Upper', 'Lower']
df1[column_list].plot(figsize=(12.2,6.4))
plt.title('Bollinger Band for Tesla Stock')
plt.ylabel('USD Price in($)')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12.2, 6.4))
ax = fig.add_subplot(1,1,1)
x_axis = df1.index
ax.fill_between(x_axis, df1['Upper'], df1['Lower'], color = 'grey')
ax.plot(x_axis, df1['Close'], color = 'gold', lw = 3, label = 'Closing Price')
ax.plot(x_axis, df1['SMA'], color = 'blue', lw = 3, label = 'Simple Moving Average')
ax.set_title('Bollinger Band for Tesla')
ax.set_xlabel('Date')
ax.set_ylabel('USD Price in ($)')
plt.xticks(rotation = 45)
ax.legend()
plt.show()

In [ ]:
new_df = df1[period-1:]
new_df

In [ ]:
def get_signal(data):
    buy_signal = []
    sell_signal = []
    
    for i in range(len(data['Close'])):
        if data['Close'][i] > data['Upper'][i]:
            buy_signal.append(np.nan)
            sell_signal.append(data['Close'][i])
        elif data['Close'][i] < data['Lower'][i]:
            buy_signal.append(data['Close'][i])
            sell_signal.append(np.nan)
        else:
            buy_signal.append(np.nan)
            sell_signal.append(np.nan)
    return(buy_signal, sell_signal)


In [ ]:
new_df['Buy'] = get_signal(new_df)[0]
new_df['Sell'] = get_signal(new_df)[1]

In [ ]:
fig = plt.figure(figsize=(12.2, 6.4))
ax = fig.add_subplot(1,1,1)
x_axis = new_df.index
ax.fill_between(x_axis, new_df['Upper'], new_df['Lower'], color = 'grey')
ax.plot(x_axis, new_df['Close'], color = 'gold', lw = 3, label = 'Closing Price', alpha = 0.35)
ax.plot(x_axis, new_df['SMA'], color = 'blue', lw = 3, label = 'Simple Moving Average', alpha = 0.35)
ax.scatter(x_axis, new_df['Buy'], color = 'green', lw = 3, label = 'Buy', marker = "^", alpha = 0.35)
ax.scatter(x_axis, new_df['Sell'], color = 'red', lw = 3, label = 'Sell', marker = "v", alpha = 0.35)
ax.set_title('Bollinger Band for Tesla')
ax.set_xlabel('Date')
ax.set_ylabel('USD Price in ($)')
plt.xticks(rotation = 45)
ax.legend()
plt.show()